In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd drive/MyDrive/chest_xray

[Errno 2] No such file or directory: 'drive/MyDrive/chest_xray'
/content/drive/MyDrive/chest_xray


# Final Project Submission
- Student Name - Vinayak Modgil
- Student pace - Full time
- Scheduled project review time/date -
- Instructor name - Yish Lim
- Plog Post URL - 

# Table of Contents
- [Introduction](#Introduction)
- [Data Collection](#Data-Collection)
- [Data Cleaning](#Data-Cleaning)
- [Data Exploration](#Data-Exploration)
- [Data Modeling](#Data-Modeling)
- [Data Interpretation](#Data-Interpretation)
- [Recommendations and Conclusions](#Recommendations-and-Conclusions)

# Introduction

# Data Collection

## Import necessary packages

In [20]:
import pandas as pd
import os, sys, glob
import cv2

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from PIL import Image

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


## Global Functions

In [23]:
def n_images_identifier(base_folder="/content/drive/MyDrive/chest_xray/"):
    '''
    returns the number of different
    types of images in the folders
    
    n_images_identifies: 
    - base folder : common folder holding all the images.
    '''
    #Takes the folder type as normal or pneumonia
    folder_type = ["NORMAL", "PNEUMONIA"]
    #
    train_folder = base_folder + "train/"
    test_folder = base_folder + "test/"
    val_folder = base_folder + "val/"
    
    # iterates for folder in train_folder, test_folder
    #and val_folder
    for folder in [train_folder, test_folder, val_folder]:
        
        #Further iterates for _type in folder_type
        for _type in folder_type:
            if _type=="NORMAL":
                n_normal = len(os.listdir(folder + _type))
            else:
                n_infected = len(os.listdir(folder + _type))
        print(f"Set: {folder}, normal images: {n_normal}, infected images: {n_infected}")
        print(f"Folder {folder} has a total of {n_normal+n_infected} images\n")
    return 
    

In [24]:
n_images_identifier()

Set: /content/drive/MyDrive/chest_xray/train/, normal images: 1341, infected images: 3875
Folder /content/drive/MyDrive/chest_xray/train/ has a total of 5216 images

Set: /content/drive/MyDrive/chest_xray/test/, normal images: 234, infected images: 390
Folder /content/drive/MyDrive/chest_xray/test/ has a total of 624 images

Set: /content/drive/MyDrive/chest_xray/val/, normal images: 8, infected images: 8
Folder /content/drive/MyDrive/chest_xray/val/ has a total of 16 images



In [41]:
def data_processing(img_dims,
                    batch_size_train=None,
                    batch_size_test=None,
                    batch_size_val=None,
                    base_folder="/content/drive/MyDrive/chest_xray/"):
    train_folder = base_folder + "train/"
    test_folder = base_folder + "test/"
    val_folder = base_folder + "val/"

    #Data genereation objects
    train_images = ImageDataGenerator(rescale=1/255)
    test_images = ImageDataGenerator(rescale=1/255)
    val_images = ImageDataGenerator(rescale=1/255)
    
    
    #Use flow from directory to get train and test 
    #images
    if batch_size_train != None and batch_size_test != None and batch_size_val != None:

      train_chest = train_images.flow_from_directory(
      directory=train_folder,
      target_size=(img_dims, img_dims),
      batch_size=batch_size_train, 
      class_mode="binary", 
      shuffle=True)
    
      test_chest = test_images.flow_from_directory(
      directory=test_folder, 
      target_size=(img_dims, img_dims),
      batch_size=batch_size_test, 
      class_mode="binary", 
      shuffle=True)

      val_chest = val_images.flow_from_directory(
      directory=val_folder,
      target_size=(img_dims, img_dims),
      batch_size = batch_size_val,
      class_mode="binary",
      shuffle=True)

      return train_chest, test_chest, val_chest

Found 5216 images belonging to 2 classes.
